In [16]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [17]:
#brings up video capturing device


#DirectShow backend API for Windows to increase performance
#cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap = cv2.VideoCapture(0)
#access the pose model : pass in confidence accordingly 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        #reads current frame
        ret, frame = cap.read()

        #recolor image -> pass rgb to media pipe 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #access pose model 
        results = pose.process(image)
        
        #recolor to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        #gives popup on screen with the current frame fed in
        cv2.imshow('Mediapipe Feed', frame)

        #close screen if user tries to hit q 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

